<a href="https://colab.research.google.com/github/FaiazS/AI-Powered-Multilingual-IT-Support-Analyst/blob/main/IT_Support_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

!pip install langchain-groq

!pip install langchain-community

import os

from google.colab import userdata

from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import JsonOutputParser

from pydantic import BaseModel, Field

In [ ]:
from langchain_groq import ChatGroq

groq_api_key = os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

llama = ChatGroq(

                 model = 'llama-3.3-70b-versatile'
)

In [ ]:
class ITSupportResolution(BaseModel):

  issue_message_original: str = Field(description="The initial query message reporting the issue.")

  language_orginal: str = Field(description= "Language in which the issue was reported in.")

  category: str = Field(description= "1 or 2 words decribing the category of which techncial area the issue belongs to")

  issue_message_translated: str =  Field(description = "Translated query message of the issue reported")

  resolution: str = Field("Resolution to the customer's issue reported in their native language - language_original")

  resolution_translated: str = Field("Resolution to the customer in English")

In [ ]:
output_parser = JsonOutputParser(pydantic_object = ITSupportResolution)

it_support_analyst_prompt ="""

Act as a seasoned Information Technology(IT) Support Analyst.

For the IT query mentioned below, incorporate the following output format when drafting the resolution.

Resolution format instruction:

{resolution_format_instruction}

IT query:

{it_query}

"""

system_prompt = PromptTemplate(

                               template = it_support_analyst_prompt,

                               input_variables=['it_query'],

                               partial_variables = {

                                   'resolution_format_instruction' : output_parser.get_format_instructions()

                                   }


)

In [ ]:
it_support_analyst_chain = system_prompt | llama | output_parser

In [ ]:
it_query_queue = [


    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

formatted_queries = [{'it_query' : query} for query in it_query_queue]

formatted_queries[1]

{'it_query': 'Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.'}

In [ ]:
llama_response = it_support_analyst_chain.map().invoke(formatted_queries)

In [ ]:
llama_response[1]

{'issue_message_original': 'Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.',
 'language_orginal': 'Italian',
 'category': 'Printing',
 'issue_message_translated': 'I have problems printing documents remotely. The job is not being sent to the network printer.',
 'resolution': 'Controlla la connessione di rete e assicurati che la stampante di rete sia configurata correttamente. Inoltre, verifica che il driver della stampante sia aggiornato e compatibile con il sistema operativo. Se il problema persiste, prova a riavviare la stampante e il computer.',
 'resolution_translated': 'Check the network connection and make sure the network printer is configured correctly. Also, verify that the printer driver is up-to-date and compatible with the operating system. If the problem persists, try restarting the printer and the computer.'}

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(llama_response)

dataframe

,issue_message_original,language_orginal,category,issue_message_translated,resolution,resolution_translated
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Contact Sync,I am unable to sync my contacts with the phone...,Verifique se o seu telefone está conectado à m...,Check if your phone is connected to the same W...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printing,I have problems printing documents remotely. T...,Controlla la connessione di rete e assicurati ...,Check the network connection and make sure the...
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer Issue,"I replaced the printer toner, but the print qu...",トナー交換後、プリンターのヘッドをクリーンしてください。また、印刷設定を確認し、適切な品質設...,"After replacing the toner, please clean the pr..."
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,I am unable to log in to the time tracking sys...,Проверьте правильность ввода логина и пароля. ...,Check the correctness of the login and passwor...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Connection,My internet connection is very slow and someti...,İnternet bağlantınızın yavaş olmasının ve kesi...,There could be several reasons why your intern...
5,Не могу установить обновление безопасности. По...,Russian,Security Update,I cannot install the security update. An error...,Попробуйте перезагрузить компьютер и попытатьс...,Try restarting your computer and attempt to in...
